In [2]:
import numpy as np

In [141]:
# Implementation for asynchronous Hopfield neural network
# Note: memory capacity ≃ 0.14*nodes
class Hopfield_Neural_Network:
    def __init__(self,nodes,iterations=100,weights=None):
        self.nodes = nodes
        self.iterations = iterations
        try:
            if weights == None:
                self.weights = np.zeros((nodes,nodes))
        except ValueError:
            self.weights = weights
    
    def store(self,input):
        dW = np.outer(input.transpose(),input)
        np.fill_diagonal(dW,0)
        self.weights += dW
        
    def recall(self,input):
        update_sequence = np.random.choice(self.nodes, self.iterations)
        for node in update_sequence:
            input[node] = np.sign(np.inner(input,self.weights[:,node]))
        return input

In [136]:
a = Hopfield_Neural_Network(10)